In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
from sklearn import metrics

In [3]:
#Get the training data
df = pd.read_csv("/kaggle/input/titanic/train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
df.dropna(inplace=True)

In [11]:
#Create Validation data
y=df["Survived"].values
features = df[["Pclass", "Sex", "Age", "SibSp", "Parch"]]
X = pd.get_dummies(features)
idx = np.random.rand(len(df)) < 0.75
X_train = X[idx]
X_validation = X[~idx]
y_train = y[idx]
y_validation = y[~idx]
print(X.head(5))
print(y[0:5])
#y = train_data["Survived"]
#X = pd.get_dummies(train_data[features])
#X_test = pd.get_dummies(test_data[features])

    Pclass   Age  SibSp  Parch  Sex_female  Sex_male
1        1  38.0      1      0           1         0
3        1  35.0      1      0           1         0
6        1  54.0      0      0           0         1
10       3   4.0      1      1           1         0
11       1  58.0      0      0           1         0
[1 1 0 1 1]


In [8]:
#Get the testing data
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
test_data.shape

(418, 11)

In [22]:
#Random Forest 
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=1)
model.fit(X_train, y_train)
predictions = model.predict(X_validation)
print("Accuracy of Random Forest is ", metrics.accuracy_score(predictions,y_validation))

Accuracy of Random Forest is  0.8148148148148148


In [35]:
X_test = pd.get_dummies(test_data[["Pclass", "Sex", "Age", "SibSp", "Parch"]])
X_test.dropna(inplace=True)
X_test_copy = pd.get_dummies(test_data[["PassengerId","Pclass", "Sex", "Age", "SibSp", "Parch"]])
X_test_copy.dropna(inplace=True)
predictions = model.predict(X_test)

In [33]:
#Save Random Forest predictions
output = pd.DataFrame({'PassengerId': X_test_copy.PassengerId, 'Survived': predictions})
output.to_csv('my_submission_RF.csv', index=False)
print("Your submission was successfully saved!")

ValueError: array length 332 does not match index length 87

In [17]:
#SVM
from sklearn import svm
clf = svm.SVC(kernel='sigmoid')
clf.fit(X_train, y_train) 
yhat = clf.predict(X_validation)
print(metrics.accuracy_score(y_validation,yhat))

0.7222222222222222


In [21]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(C=0.01, solver='newton-cg').fit(X_train,y_train)
LR_y = LR.predict(X_validation)
print(metrics.accuracy_score(y_validation,LR_y))

0.7407407407407407
